This notebook also is based on a kaggle dataset of different species of Beetles, in which all the code and the neural network system is analyzed in a research project. Download the dataset to your google drive and for all pathways, derive them from your own drive, as the ones in the notebook are specific to their location in my drive. 

Link to the dataset: https://www.kaggle.com/general/189980 

Link to the research project: https://docs.google.com/document/d/1yjy33XpDlXLgent6nDCLUkBmveFCA_NB/edit 

In [ ]:
#@title Imports
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%pylab inline
from sklearn import model_selection
from PIL import Image
import glob
!pip install pygbif
from PIL import Image
import os
import pygbif
from pygbif import species
import numpy as np
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, InputLayer
from keras.layers import Activation, MaxPooling2D, Dropout, Flatten, Reshape
from keras.utils.np_utils import to_categorical 
from keras.wrappers.scikit_learn import KerasClassifier
import cv2
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from collections import Counter
import tensorflow.keras as keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, InputLayer
from keras.layers import Activation, MaxPooling2D, Dropout, Flatten, Reshape
from keras.utils.np_utils import to_categorical 
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd
import seaborn as sns
!pip install keras-tuner -q
from tensorflow import keras
import tensorflow as tf
import keras_tuner as kt
hp = kt.HyperParameters()
from keras_tuner.tuners import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters
from numpy import asarray
from numpy import savetxt

In [ ]:
#@title Mount Notebook to Google Drive
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
#@title Plotting 3 Images
pathways = ['/content/gdrive/MyDrive/Insect Data/Beetles/1035167/d162s0011.jpg', 
            '/content/gdrive/MyDrive/Insect Data/Beetles/1035185/d158s0001.jpg', 
            '/content/gdrive/MyDrive/Insect Data/Beetles/1035194/d170s0005.jpg'] 
for i in pathways:
  img = Image.open(i)
  imgplot = plt.imshow(img)
  plt.show()



In [ ]:
#@title Verancular Name
for file_path in glob.glob('//content/gdrive/MyDrive/Insect Data/Beetles/*'): 
    filename=os.path.basename(file_path)
    try:
      print(filename, 'has a vernacular name of', species.name_usage(key=filename, data='vernacularNames', limit=1)['results'][0]['vernacularName'])
    except:
      print(filename, 'does not have vernacular name')


In [ ]:
#@title Species Name
j = 0
for file_path in glob.glob('//content/gdrive/MyDrive/Insect Data/Beetles/*'): 
    filename=os.path.basename(file_path)
    print(filename, 'has a species name of', species.name_usage(key=filename)['species'])
    if j > 50:
      break



In [ ]:
#@title Creating experimental data set and labels
image_list = []
image_name = []
file_dr_list = []
num_dirs = 0
for file_dr in glob.glob('//content/gdrive/MyDrive/Insect Data/Beetles/*'):
    num_images = 0
    print(file_dr)
    file_dr_list.append(file_dr)
    for filename in glob.glob(f'{file_dr}/*.*'):
        image_list.append(filename)
        image_name.append(file_dr)
        num_images+=1
        if num_images >=200:
          break
    num_dirs+=1
    if num_dirs >19:
        break
with open('/content/gdrive/My Drive/array.csv', 'w') as f:
    np.savetxt(f, file_dr_list, fmt="%s")


In [ ]:
#@title Resizing Images
pathways = ['/content/gdrive/MyDrive/Insect Data/Beetles/1035167/d162s0011.jpg']            
img = Image.open(im) 
imgplot = plt.imshow(img)
plt.show() 
image = img.resize((32,32)) 
imgplot = plt.imshow(image)
plt.show()

In [ ]:
#@title Grayscale
image = '/content/gdrive/MyDrive/Insect Data/Beetles/1035167/d162s0011.jpg'
img = Image.open(image)
imgplot = plt.imshow(img)
plt.show()
img = Image.open(image).convert('L')
imgplot = plt.imshow(img)
plt.show()
imgplot = plt.imshow(img, cmap='gray')
plt.show()


In [ ]:
#@title normalizing images
def normalize_one_image(image):
  return (image - np.min(image)) / (np.max(image) - np.min(image))
image = '/content/gdrive/MyDrive/Insect Data/Beetles/1035167/d162s0011.jpg'
img = Image.open(image)
normalize_one_image(img)

In [ ]:
#@title Applying preprocessing
plt.show()
Beetleimg_list = []
for image in image_list:
  img = Image.open(image).convert('L')
  image = img.resize((32,32))
  Beetleimg_list.append(normalize_one_image(image))

In [ ]:
#@title Arraying for 2 categories
def categorical_to_numpy(labels_in):
  labels = []
  for label in labels_in:
    if label == '//content/drive/MyDrive/Insect Data/Beetles/6097892':
      labels.append(np.array([1, 0]))
    else:
      labels.append(np.array([0, 1]))
  return np.array(labels)
Beetleimg_list = np.array(Beetleimg_list)
Beetleimg_list = Beetleimg_list.astype(float)
image_name = categorical_to_numpy(image_name)

In [ ]:
#@title Arraying for 20 categories
image_names = []
dict_ = {}
j=0
for i in file_dr_list:
  dict_.update({i:j})
  j=j+1
for label in image_name:
  l = np.zeros(20)
  l[dict_.get(label)]=1 #insert 1 into position in l conver value in dictionary to 1
  image_names.append(l)
Beetleimg_list = np.array(Beetleimg_list)
Beetleimg_list = Beetleimg_list.astype(float)


In [ ]:
#@title Train test split
inputs_train, inputs_test, labels_train, labels_test = model_selection.train_test_split(Beetleimg_list, image_names, test_size=0.2)
labels_test = np.array(labels_test)
labels_train = np.array(labels_train)

In [ ]:
#@title Classifying 2 images
def CNNClassifier(num_epochs=2, layers=1, dropout=0.15):
  def create_model():
  
    model = Sequential()
    model.add(Reshape((32, 32, 1)))
    
    for i in range(layers):
      model.add(Conv2D(32, (3, 3), padding='same'))
      model.add(Activation('relu'))
    
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(dropout))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(dropout))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(dropout))
    model.add(Dense(2))
    model.add(Activation('softmax'))

    # initiate RMSprop optimizer
    opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

    # Let's train the model using RMSprop
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt, #change to opt
                  metrics=['accuracy'])
    return model
  return KerasClassifier(build_fn=create_model, epochs=num_epochs, batch_size=10, verbose=2)

cnn = CNNClassifier()
cnn.fit(inputs_train, labels_train)
predictions = cnn.predict(inputs_test)
print (cnn.score(inputs_test, labels_test))

In [ ]:
#@title Classifying 20
def CNNClassifier(num_epochs=20, layers=1, dropout=0.15, kernel_size=3, pooling_size = 2):
  def create_model():
  
    model = Sequential()
    model.add(Reshape((32, 32, 1)))
    
    for i in range(layers):
      model.add(Conv2D(32, (kernel_size), padding='same'))
      model.add(Activation('relu'))
    
    model.add(Conv2D(32, (kernel_size)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(pooling_size)))
    model.add(Dropout(dropout))

    model.add(Conv2D(64, (kernel_size), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (kernel_size)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(pooling_size)))
    model.add(Dropout(dropout))

    #units = hp.Int("units", min_value=32, max_value=512, step=32)
    #activation = hp.Choice("activation", ["relu", "tanh"])
    #drpout = hp.Boolean("dropout")
    #lr = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")

    model.add(Flatten())
    model.add(Dense(96)) 
    model.add(Activation('relu'))
    model.add(Dropout(dropout))
    model.add(Dense(20))
    model.add(Activation('softmax'))

    # initiate RMSprop optimizer
    opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)
    # Let's train the model using RMSprop
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt, #change to opt
                  metrics=['accuracy'])
   
    return model
  return KerasClassifier(build_fn=create_model, epochs=num_epochs, batch_size=10, verbose=2)

cnn = CNNClassifier()
history = cnn.fit(inputs_train, labels_train, validation_data=(inputs_test, labels_test), epochs=100)

In [ ]:
#@title hyperparameters
def CNNClassifier(hp):#
    num_epochs=20
    layers=1
    dropout=0.15
    kernel_size=3
    pooling_size = 2

    model = Sequential()
    model.add(Reshape((32, 32, 1)))
    
    for i in range(hp.Int("num_layers", 1, 3)):
      model.add(Conv2D(32, (kernel_size), padding='same'))
      model.add(Activation('relu'))

    first_conv2d = hp.Int("2nd_conv2d", min_value=32, max_value=512, step=32)
    model.add(Conv2D(first_conv2d, (kernel_size)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(pooling_size)))
    model.add(Dropout(dropout))

    Second_conv2d = hp.Int("3rd_conv2d", min_value=32, max_value=512, step=32)
    model.add(Conv2D(Second_conv2d, (kernel_size), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (kernel_size)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(pooling_size)))
    model.add(Dropout(dropout))


    model.add(Flatten())
    units = hp.Int("units_0", min_value=32, max_value=512, step=32)
    model.add(Dense(units))
    activation = hp.Choice("activation", ["relu", "tanh"])

    model.add(Activation(activation))
    #drpout = hp.Boolean("dropout")
    model.add(Dropout(dropout))
    model.add(Dense(20))
    model.add(Activation('softmax'))

    # initiate RMSprop optimizer
    #opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)
    # Let's train the model using RMSprop
    lr = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(loss='categorical_crossentropy',
                  optimizer=keras.optimizers.Adam(learning_rate=lr), #change to opt
                  metrics=['accuracy'])
    return model

CNNClassifier(hp)
tuner = kt.RandomSearch(
    hypermodel=CNNClassifier,
    objective="val_accuracy",
    max_trials=10,
    executions_per_trial=1,
    overwrite=True,
)
tuner.search_space_summary()
tuner.search(inputs_train, labels_train, epochs=40, validation_data=(inputs_test, labels_test))
models = tuner.get_best_models(num_models=2)
best_model = models[0]
best_model.build(input_shape=(None, 32, 32))
tuner.results_summary()
#cnn = CNNClassifier()
#history = cnn.fit(inputs_train, labels_train, validation_data=(inputs_test, labels_test), epochs=10)
#cnn.fit(inputs_train, labels_train)
#predictions = cnn.predict(inputs_test)
#print (cnn.score(inputs_test, labels_test))
#plot_acc(history)

In [ ]:
#@title Most Optimal
def CNNClassifier(num_epochs=20, layers=3, dropout=0.15, kernel_size=5, pooling_size = 2):
  def create_model():
  
    model = Sequential()
    model.add(Reshape((32, 32, 1)))
    
    for i in range(layers):
      model.add(Conv2D(32, (kernel_size), padding='same'))
      model.add(Activation('relu'))
    
    model.add(Conv2D(320, (kernel_size)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(pooling_size)))
    model.add(Dropout(dropout))

    model.add(Conv2D(64, (kernel_size), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (kernel_size)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(pooling_size)))
    model.add(Dropout(dropout))

    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation('tanh'))
    model.add(Dropout(dropout))
    model.add(Dense(20))
    model.add(Activation('softmax'))

    # initiate RMSprop optimizer
    opt = keras.optimizers.RMSprop(learning_rate=0.00031096248743627106, decay=1e-6)
    # Let's train the model using RMSprop
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt, #change to opt
                  metrics=['accuracy'])
   
    return model
  return KerasClassifier(build_fn=create_model, epochs=num_epochs, batch_size=10, verbose=2)

cnn = CNNClassifier()
history = cnn.fit(inputs_train, labels_train, validation_data=(inputs_test, labels_test), epochs=100)

In [ ]:
#@title Displaying accuracy data.
def plot_acc(history, ax = None, xlabel = 'Epoch #'):
    history = history.history
    history.update({'epoch':list(range(len(history['val_accuracy'])))})
    history = pd.DataFrame.from_dict(history)

    best_epoch = history.sort_values(by = 'val_accuracy', ascending = False).iloc[0]['epoch']

    if not ax:
      f, ax = plt.subplots(1,1)
    sns.lineplot(x = 'epoch', y = 'val_accuracy', data = history, label = 'Validation', ax = ax)
    sns.lineplot(x = 'epoch', y = 'accuracy', data = history, label = 'Training', ax = ax)
    ax.axhline(0.05, linestyle = '--',color='red', label = 'Chance')
    ax.axvline(x = best_epoch, linestyle = '--', color = 'green', label = 'Best Epoch')  
    ax.legend(loc = 7)    
    ax.set_ylim([0, 1])

    ax.set_xlabel(xlabel)
    ax.set_ylabel('Accuracy (Fraction)')
    
    plt.show()

plot_acc(history)

In [ ]:
#@title Displaying predictions for classification of 2 images
for i in range(10):
  print(labels_train[i])
  if labels_train[i][0]:
    print('Actual: 4472907')
  else: 
    print('Actual: 6097892')
  x =  cnn.predict(inputs_train[0][np.newaxis, :, :])
  if x == 1:
    print('Predictd: 6097892')
  else:
    print('Predicted: 4472907')

In [ ]:
#@title Displaying predictions for classification of 20 images
file_dr_list = np.array(file_dr_list)
for i in range(10):
  position = labels_test[i].tolist().index(1)
  actual_filename = os.path.basename(file_dr_list[position])
  print('Actual: ', species.name_usage(key=actual_filename)['species'])
  prediction = cnn.predict(inputs_test[i][np.newaxis, :, :])
  new = (','.join(file_dr_list[prediction]))
  predicted_filename = os.path.basename(new)
  print('Prediction :', species.name_usage(key=predicted_filename)['species'])
  print('     ')